In [ ]:
import os
from dotenv import load_dotenv
from google import genai
from google.genai.types import *
import gradio as gr

In [ ]:
load_dotenv()

API_KEY = os.getenv(key="GEMINI_API_KEY")

if API_KEY:
    print("API_KEY found.")
else:
    raise "API Key not found."

gemini = genai.Client(api_key=API_KEY)

In [ ]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}


def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
    },
}

In [ ]:
tools = {"function_declarations": [price_function]}

def handle_tool_call(tool_call):
    city = tool_call.args.get("destination_city")
    return get_ticket_price(city)

In [ ]:
def get_chat_item(role, prompt):
    return {"role": role, "parts": [{"text": prompt}]}


def get_function_item(tool_call):
    return {"role": "model", "parts": [{"function_call": tool_call}]}


def generate_content(contents, system_prompt):
    response = gemini.models.generate_content(
        model="gemini-2.0-flash",
        contents=contents,
        config={"system_instruction": {"text": system_prompt}, "tools": [tools]},
    )

    while response.candidates[0].content.parts[0].function_call:
        tool_call = response.candidates[0].content.parts[0].function_call
        function_response = handle_tool_call(tool_call)

        contents.append(get_function_item(tool_call))
        contents.append(get_chat_item("user", function_response))

        response = gemini.models.generate_content(
            model="gemini-2.0-flash",
            contents=contents,
            config={
                "system_instruction": {"text": system_prompt},
                "tools": [tools],
            },
        )

    return response.candidates[0].content.parts[0].text

In [ ]:
def chat(user_prompt, history):
    system_message = "You are a helpful assistant for an Airline called FlightAI. Give short, courteous answers, no more than 1 sentence. Always be accurate. If you don't know the answer, say so."

    contents = []
    for item in history:
        contents.append(get_chat_item(item["role"], item["content"]))
    contents.append(get_chat_item("user", user_prompt))

    return generate_content(contents, system_message)

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()